# Spatial display for Wetlands Insight Tool results <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatibility:** Notebook currently compatible with only the `NCI VDI` environment


* **Special requirements:** 
    * If running on the [NCI](https://nci.org.au/), ensure that `module load dea` is   run prior to launching this notebook
    * Check you have the latest version of the `wit_tooling package` by 
      copying and pasting the following code into a cell below and running the cell
    `!pip install --user git+git://github.com/GeoscienceAustralia/wit_tooling`
      
      
* **Products used:** 
    * Collection 2 Landsat Surface Reflectance: 
    [ls5_nbart_albers](https://explorer.dea.ga.gov.au/ls5_nbart_albers),
    [ls7_nbart_albers](https://explorer.dea.ga.gov.au/ls7_nbart_albers),
    [ls8_nbart_albers](https://explorer.dea.ga.gov.au/ls8_nbart_albers)
    * Collection 2 Landsat Fractional Cover, 
    generated using the Joint Remote Sensing Research Program algorithm: 
    [ls5_fc_albers](https://explorer.dea.ga.gov.au/ls5_fc_albers),
    [ls7_fc_albers](https://explorer.dea.ga.gov.au/ls7_fc_albers),
    [ls8_fc_albers](https://explorer.dea.ga.gov.au/ls8_fc_albers)
    * Water Observations from Space, 
    generated using the Geoscience Australia Algorithm:
    [wofs_albers](https://explorer.sandbox.dea.ga.gov.au/wofs_albers)



## Background
The Spatial Wetlands Insight Tool is a tool in development to display the coverage of water, "wetness" and vegetation fractional cover in a wetland spatially. It is generated off existing Wetlands Insight Tool temporal runs. 

## Description
This notebook uses an existing Wetlands Insight Tool temporal plot, 
generated from an existing WIT run, to create a spatial plot of water, "wetness", green/photosynthetic vegetation, dry/non-photosynthetic vegetation, and bare soil for a chosen observation date. 

1. First we load the existing WIT data from either: 
    * a saved csv location
    * a shapefile to retrieve the existing WIT data from the database of previous runs
    * a csv from an Amazon s3 data bucket
2. Then we choose a time of interest to plot Spatial WIT
3. Finally we output Spatial WIT to a file for each cover type
***

Requirements - A checklist to remind us if we tick all the boxes
---------------------
- [ ] Make a nice plot to select a time/ period of interest
- [ ] Run WIT on a per-pixel basis
- [ ] Return Water/Wet/FC percentage per pixel
- [ ] Plot and output WIT spatially, with FC percentage represented as an alpha % for the colour
- [ ] Output the results as a ArcGIS-compliant Geotiff (uint8), with the shapefile name and the date in the filename
***

Functions 
---------
`bokeh wit plot`
to do a stack plot of wit data with bokeh

input: DataFrame

output: stack plot of wit data

`load_wit_data(**kwargs)` to load data

input: csv file or poly_id in database

output: DataFrame

`load_wofs_fc(query)` to load the data 

input: a query dictionary with time and geometry

output: an xarray with water/wet/FC percentage

`plot_spatial_wit(input_pixels_array)` to plot spatially

input: an xarray with water/wet/FC percentage

output: 2 dimensional plot of input

`write_geotiff(input_pixels_array, file_name)` to output to file

input: an xarray with water/wet/FC percentage

a string as file name

output: a geotiff file with input file name
***

Before running this notebook:
-----------------------------
* Follow the instructions under `Special Requirements` above to load `dea` and install `wit_tooling`

## Load packages

In [1]:
# import all the necessary packages in this cell
import pandas as pd
import numpy as np
import xarray as xr
import fiona
import yaml
from datacube import Datacube
from datacube.utils.cog import write_cog
from datetime import datetime, timedelta
import matplotlib.dates as mdates

from bokeh.io import curdoc, output_notebook, show, push_notebook
from bokeh.layouts import layout, column, row, WidgetBox, gridplot
from bokeh.models import (CheckboxGroup, Select,  CategoricalColorMapper, ColumnDataSource,HoverTool, Label,
                          SingleIntervalTicker, Slider, DatetimeTickFormatter, YearsTicker, Legend, TapTool,
                          CustomJS, LegendItem, field, Range1d)
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.glyphs import Text
from bokeh.models.tickers import DatetimeTicker
from bokeh.models import LinearColorMapper
from bokeh.colors import RGB
from bokeh.plotting import figure

from datacube.virtual.impl import VirtualDatasetBox
from datacube.virtual import construct
from datacube.utils.geometry import CRS, Geometry
from shapely.geometry import mapping, box
from enum import Enum
import os, sys, urllib, logging
import seaborn as sns
import matplotlib.pyplot as plt
import ssl

from wit_tooling import query_wit_data, load_timeslice, convert_shape_to_polygon, generate_raster

ModuleNotFoundError: No module named 'wit_tooling'

In [ ]:
output_notebook()
ssl._create_default_https_context = ssl._create_unverified_context

_LOG = logging.getLogger(__name__)
stdout_hdlr = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('[%(asctime)s.%(msecs)03d - %(levelname)s] %(message)s')
stdout_hdlr.setFormatter(formatter)
_LOG.addHandler(stdout_hdlr)
_LOG.setLevel(logging.DEBUG)

### Global variables
If you are using a shapefile, csv file, or Amazon s3 link to the existing WIT run, 
the path must be set in the cell below this cell:

* `shapefile`: NCI path to shapefile 
(e.g. `'/g/data1a/r78/DEA_Wetlands/shapefiles/ramsar_wetlands_3577_20190403.shp'`). 
You must have permissions to the project directory,
and the shapefile must be in [Australian Albers EPSG 3577 projection](https://spatialreference.org/ref/epsg/gda94-australian-albers/)
* `csv_file`: NCI path to WIT results CSV (e.g. `'/g/data1a/u46/users/ea6141/dea-notebooks/Spatial_WIT/sample_data/Western Port_Western Port_VIC_19.csv'`)
* `pd_yaml`: Yaml file necessary to generate WIT 
e.g. `'/g/data/u46/users/ea6141/wit_tooling/aux/fc_pd.yaml'`). 
Specifies input datasets.
* `s3_url`: Amazon s3 url link to pre-generated WIT csvs folder 
(e.g. `'https://dea-public-data-dev.s3-ap-southeast-2.amazonaws.com/Wetlands_Insight_Tool/WIT_v3'`). 


In [ ]:
# put global variables in this cell

shapefile = '/g/data1a/r78/DEA_Wetlands/shapefiles/ramsar_wetlands_3577_20190403.shp'
csv_file = '/g/data1a/u46/users/ea6141/dea-notebooks/Spatial_WIT/sample_data/Western Port_Western Port_VIC_19.csv'
pd_yaml = '/g/data/u46/users/ea6141/wit_tooling/aux/fc_pd.yaml'
s3_url = 'https://dea-public-data-dev.s3-ap-southeast-2.amazonaws.com/Wetlands_Insight_Tool/WIT_v3'

Functions used in this notebook to create, display and export Spatial WIT
----------------------------------------------


In [ ]:
def bokeh_WIT_plot(WITdata, polyName='provided polygon'):
    '''
    last modified: May 2020
    
    Parameters
    ----------
    WITdata : xarray data array produced by load_wit_data function
    polyName : string
               A name for the polygon to identify the plot, optional. Defaults to 'provided polygon' 
                   
    Returns
    -------
    A bokeh stack plot of the contents of the vector file in water, wet, green, dry and bare. Plot can be zoomed in to select a date. 
    '''
    
    #set up color palate for bokeh WIT plot
    pal = [sns.xkcd_rgb["cobalt blue"],
           sns.xkcd_rgb["neon blue"],
           sns.xkcd_rgb["grass"],
           sns.xkcd_rgb["beige"],
           sns.xkcd_rgb["brown"]]  

    #these are tools we want to use in the plot
    TOOLS = ["pan, wheel_zoom, box_zoom, reset, tap, save"]





 


    return p


In [ ]:
def plot_spatial_WIT(spatial_wit_xr):
    """
        plot spatial wit
        input:
            an xarray of spatial wit
        output:
            figure from bokeh
    """
    image_list = [spatial_wit_xr[var].data[0] for var in spatial_wit_xr.data_vars]
    # all below is to setup the pallete
    transparent_white = RGB(255, 255, 255, 0)
    colbat_blue = [RGB(3, 10, 167, 1)]
    neon_blue = [RGB(4, 217, 255, 1)]
    grass_green = [RGB(63, 155, 11, t) for t in np.arange(0.1, 1, 0.1)]
    soil_brown = [RGB(96, 70, 15, t) for t in np.arange(0.1, 1, 0.1)]
    dry_biege = [RGB(230, 218, 166, t) for t in np.arange(0.1, 1, 0.1) ]
    var_colors = [soil_brown, grass_green, dry_biege, neon_blue, colbat_blue]
    color_map = [LinearColorMapper([transparent_white]+c, low=0, high=100,
                                   nan_color=transparent_white) for c in var_colors]
    
    # do the image plot
    p =figure(plot_width=900, plot_height = 900,
             tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])

    for i in range(5):
        p.image(image=image_list[i:i+1], x=fc_wofs_data.x.data.min(), y=fc_wofs_data.y.data.max(),
            dh=(fc_wofs_data.x.data.max() - fc_wofs_data.x.data.min()),
            dw=(fc_wofs_data.y.data.max() - fc_wofs_data.y.data.min()),
            color_mapper = color_map[i])
    # to do
    # legend, title, tooltip to show rignt value bla...
    return p

In [ ]:
def save_geotiff(spatial_wit_xr, filename):
    """
        save spatial WIT result to geotiffs, each band will be output to individual tiff
        input:
            an xarray Dataset of spatial WIT
        output:
            multiple cloud-optimized geotiffs (cogs) on disk
    """
    for var in spatial_wit_xr.data_vars:
        
        #create file name per band
        band_output = file_name + "_" + var + ".tif"
        #get spatial attributes from the parent dataset
        geotiff_out_wit = spatial_wit_xr[var]
        geotiff_out_wit.attrs = spatial_wit_xr.attrs
        write_cog(geotiff_out_wit, band_output, blocksize=16)
    return

In [ ]:
def rename_columns(wit_df):
    """
        Rename and reindex the input DataFrame
        input: 
            loaded wit data as pandas DataFrame
        output:
            renamed and reindexed pandas DataFrame
    """
    #give the index a name that reflects that it is time, measured in UTC not AEDT/AEST
    wit_df = wit_df.set_index('TIME')
    wit_df.index.name = 'utc_time'
    #format the index of the dataframe as a date, not as a string
    wit_df.index = pd.to_datetime(wit_df.index)
    #Rename the columns so they are easier to understand and plot
    wit_df = wit_df.rename(columns={"WATER" : "water", 
                            "WET" : "wet",
                           "PV" : "green",
                           "NPV" : "dry",
                           "BS" : "bare"}) 
    #converting to percentages to make plotting easier
    #first convert if not already a percentage
    if wit_df.max().max() <=1.0:
        wit_df = wit_df*100
    #WITdata.head()
    return wit_df

In [ ]:
def load_wit_data(**kwargs):
    """
        Load pre-computed wit data from 3 different sources with the given parameter. Source is chosen by the key
        in kwargs.
        input parameters:
            csv = csv_path: csv file path
            shape = a shape from shape file
            s3_url = url of s3 bucket: s3 bucket path
        output:
            pandas dataframe of wit data
    """
    if kwargs.get("csv") is not None:
        wit_data = pd.read_csv(kwargs['csv'])
    elif kwargs.get('shape') is not None:
        _, wit_data = query_wit_data(kwargs['shape'])
        wit_data = pd.DataFrame(data=wit_data, columns=['TIME', 'BS', 'NPV', 'PV', 'WET', 'WATER'])
    elif kwargs.get('s3_url') is not None:
        wit_data = pd.read_csv(kwargs['s3_url'], infer_datetime_format=True)
    return wit_data

In [ ]:
# next three functions are used to load fc and wofs data with give geometry and time
def construct_product(product_yaml):
    """
        Construct a virtual product with the given yaml file
        input:
            product_yaml: the yaml file path
        output:
            virtual product instance
    """
    with open(product_yaml, 'r') as f:
        recipe = yaml.safe_load(f)
    fc_product = construct(**recipe)
    return fc_product

def query_datasets(fc_product, shape, crs, time_range):
    """
        Query the datasets in datacube database with the given shape and time period
        input:
            fc_product: virtual product instance
            shape: a shape from shape file
            crs: crs string from shape file
            time_range: a tuple of (start_time, end_time)
        output:
            grouped datasets: VirtualDatasetBox
    """
    dc = Datacube()
    query_poly = convert_shape_to_polygon(shape['geometry'])
    query_poly = Geometry(mapping(box(*query_poly.bounds)), CRS(crs))
    query = {'geopolygon': query_poly, 'time': time_range}
    datasets = fc_product.query(dc, **query)
    grouped = fc_product.group(datasets, **query)
    return grouped

def load_wofs_fc(fc_product, grouped, time_slice):
    """
        Load cloud free wofs, TCW and FC data with the given time or a tuple of (start_time, end_time)
        input:
            fc_product: virtual product instance
            grouped: grouped datasets
            time_slice: a single time or tuple of (start_time, end_time)
        output:
            wofs, TCW and FC data: xr.Dataset
    """
    if not (isinstance(time_slice, list) or isinstance(time_slice, tuple)):
         time_slice = [time_slice]
    to_load = VirtualDatasetBox(grouped.box.loc[time_slice], grouped.geobox,
                grouped.load_natively, grouped.product_definitions, grouped.geopolygon)
    fc_wofs_data = load_timeslice(fc_product, to_load)
    return fc_wofs_data

In [ ]:
def spatial_wit(fc_wofs_data, mask):
    """
        Compute spatial wit with wofs, TCW and FC data with the given polygon mask
        input:
            fc_wofs_data: wofs, TCW and FC data: xr.Dataset
            mask: a polygon mask: np.array
        output:
            spatial wit results: xr.Dataset
    """
    none_water_vars = list(fc_wofs_data.data_vars)[:-1]
    water_var = list(fc_wofs_data.data_vars)[-1]
    fc_data = fc_wofs_data[none_water_vars].where(fc_wofs_data[water_var] < 1, 0)
    tcw_percent = fc_data['TCW'] >= -350
    fc_percent = fc_data.drop('TCW').where(~tcw_percent, 0)
    fc_wofs_perc = xr.merge([fc_percent, (tcw_percent.astype("int") * 100),
                             (fc_wofs_data[water_var].astype("int") * 100)])
    fc_wofs_perc = fc_wofs_perc.where(mask == int(shape['id']), -127).astype("int16")
    fc_wofs_perc.attrs.update(fc_wofs_data.attrs)
    for var in fc_wofs_perc.data_vars:
        fc_wofs_perc[var].attrs['nodata'] = -127
    return fc_wofs_perc

## Main : 
here we run functions and produce outputs
***

## Load WIT data using one of the methods in the cell below

In [ ]:
# load wit data from database with a chosen shape
with fiona.open(shapefile) as allshapes:
    shape_crs = allshapes.crs_wkt
    shape = next(iter(allshapes))
    wit_data = load_wit_data(shape=shape)

# or load from s3 bucket
# s3_filename = 'Kerang%20Wetlands_Hird%20Swamp_VIC_17.csv'
# wit_data = load_wit_data(s3_url='/'.join([s3_url, s3_filename]))

# or load from local csv
# wit_data = load_wit_data(csv=csv_file)

wit_data = rename_columns(wit_data)

In [ ]:
#print some details about the shapefile. 
#You will have to change this for other shapefiles
print(f"loaded shape id {shape['id']},\
 {shape['properties']['RAMSAR_NAM']}, {shape['properties']['WETLAND_NA']}")
wit_data.head()

#---working on stack plot here ---#

In [ ]:
wit_data.index[0].to_datetime64()

In [ ]:
wit_data.index

In [ ]:
WITdata = wit_data
polyName = "fixme"

from bokeh.models import ColumnDataSource, HoverTool, TapTool, WheelZoomTool, ResetTool, PointDrawTool, BoxZoomTool, SaveTool
from bokeh.events import Tap
import datetime
from bokeh.plotting import figure


#try setting the data source explicitly
source = ColumnDataSource(data= WITdata)
#timesource =ColumnDataSource(data=WITdata.index) -thinking of making new dataframe just for time for the tap tool

#set up color palate for bokeh WIT plot
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]  

#lets put a title on the plot
title =f'Percentage of area dominated by WOfS, Wetness, Fractional Cover for {polyName}'    

#attempt to get the hovertool running
#turn the hover tool off for the slc-off rectangle as it displays information differently




#set up the x axis to recognise date and time. Note that you will only see the days when you zoom in.
p =figure(plot_width=1200, 
          plot_height = 400, 
          x_axis_type='datetime',
         title=title, tools=[ResetTool(), BoxZoomTool(dimensions="width")])

# Configure a renderer to be used upon hover
hover_glyph = p.circle(x='utc_time', source=source,
                         size=15, alpha=0,
                         hover_fill_color='black', hover_alpha=0.5)

hover = HoverTool(names = ["lineplot"],
                  tooltips = [
    ("observation", "$index"),
    ("date", "$x{%d/%m/%Y}"),
    ("bare","@bare{0.0}%"),
    ("dry", "@dry{0.0}%"),
    ("green","@green{0.0}%"),
    ("wet","@wet{0.0}%"),
    ("water","@water{0.0}%")],     
                formatters=
    {"$x":"datetime"})#,
           #       renderers=[hover_glyph]
#)


#trialling different ways of getting the tools to work. Both adding tools and including in the figure work.
p.add_tools(hover, TapTool(), WheelZoomTool(), SaveTool())


p.sizing_mode = "scale_width"

#align the title in the centre
p.title.align= "center"
p.title.text_font_size="12pt"

#label axes
p.yaxis.axis_label=("percentage of polygon classified as type")
p.yaxis.axis_label_text_font_size="8pt"

#we need screen units to put the attribution label under the plot. Don't ask why.
label_opts = dict(
    x=0, 
    y=0,
    x_units='screen', 
    y_units='screen',
    text_font_style="italic", 
    text_font_size="8.5pt")

#underplot context
msg1 = 'The Fractional Cover algorithm developed by the Joint Remote Sensing Research Program\n\
and the Water Observations from Space algorithm developed by Geoscience Australia are used in the production of this data'
caption1 = Label(text=msg1, **label_opts)

p.add_layout(caption1, 'below')

p.xaxis.formatter=DatetimeTickFormatter(years =["%Y"], months=["%m/%Y"] ,days=["%d/%m/%Y"])
p.xaxis.major_label_orientation = 45

#create the actual stack plot using data from the pandas dataframe 
p.varea_stack(['water', 
              'wet',
              'green',
              'dry',
              'bare'], x= 'utc_time', name = "stackplot", color=pal, fill_alpha=0.7, source = source, 
              legend_label=["water","wet","green","dry","bare"], muted_color="grey", muted_alpha=0.2)

#duplicate the stack plot as a line plot in order to get the hover tool working 
p.vline_stack(['water', 
              'wet',
              'green',
              'dry',
              'bare'], x= 'utc_time', name="lineplot", color=pal, line_alpha=0.7, source = source,)# active_tap = Instance)???

   
#set the new WIT graph ranges.
left, right, bottom, top = WITdata.index[0], WITdata.index[-1], 0, 100 #set 
p.x_range=Range1d(left, right)
p.y_range=Range1d(bottom, top)
p.xaxis.bounds=(left,right)
p.yaxis.bounds=(bottom,top)

#now we want to overplot the data on the plot
#create rectangle borders for no-data times (SLC-off only)
LS5_8_gap_start = datetime.datetime(2011,11,1)
LS5_8_gap_end = datetime.datetime(2013,4,1)

#plot our dead satellite rectangle
p.hbar(y=50, 
       height=100,
       left=LS5_8_gap_start, 
       right=LS5_8_gap_end, 
       name ="LS7 SLC-OFF",
       color="white", 
       alpha=0.5, 
       hatch_color="white", 
       hatch_pattern='/',
       hatch_alpha=0.6,
       line_color="white",
       line_width =2,
       line_alpha=0.6)

p.legend
p.legend.location="bottom_left"
p.legend.click_policy="mute"
p.legend.background_fill_alpha=0.5
p.legend.border_line_alpha=0.5
p.legend.label_text_font_size="9pt" 

#reverse the legend 
p.legend[0].items.reverse()


taptool = p.select(type=TapTool)
taptool.callback = print(source.selected)

show(p)


In [ ]:
# Stack plot of wit
plot = bokeh_WIT_plot(wit_data)
show(plot)

### Get the location of all the time slices that could potentially be used to calculate Spatial WIT

In [ ]:
# it's helpful to get the location of data rather than load them
# and it will save you time without querying database multiple times
time_range = (wit_data.index.min(), wit_data.index.max())
#build a product for our data using the yaml file to specify which datasets we need
fc_product = construct_product(pd_yaml)
datasets = query_datasets(fc_product, shape, shape_crs, time_range)
_LOG.debug("Query datasets %s", datasets)

### Decide which observation in time to plot in space

In [ ]:
# then decide which time slice(s) you want to load
# e.g. 1988-02-18 23:13:23.000  in wit_data concerns you
time_slice = np.datetime64(wit_data.index[4])
_LOG.debug("load time slice %s", time_slice)
fc_wofs_data = load_wofs_fc(fc_product, datasets, time_slice)
# mask by the geometry of given polygon
# first parameter of generate_raster is a tuple of (shape geometry, [integer of shape id])
mask = generate_raster([(shape['geometry'], int(shape['id']))], datasets.geobox)
fc_wofs_perc = spatial_wit(fc_wofs_data, mask)

In [ ]:
# plot the spatial WIT
plot = plot_spatial_WIT(fc_wofs_perc)
show(plot)

In [ ]:
fc_wofs_perc.PV

In [ ]:
spatial_wit_xr = fc_wofs_perc
image_list = [spatial_wit_xr[var].data[0] for var in spatial_wit_xr.data_vars]
# all below is to setup the pallete
transparent_white = RGB(255, 255, 255, 0)
colbat_blue = [RGB(3, 10, 167, 1)]
neon_blue = [RGB(4, 217, 255, 1)]
grass_green = [RGB(63, 155, 11, t) for t in np.arange(0.1, 1, 0.1)]
soil_brown = [RGB(96, 70, 15, t) for t in np.arange(0.1, 1, 0.1)]
dry_biege = [RGB(230, 218, 166, t) for t in np.arange(0.1, 1, 0.1) ]
var_colors = [soil_brown, grass_green, dry_biege, neon_blue, colbat_blue]
color_map = [LinearColorMapper([transparent_white]+c, low=0, high=100,
                               nan_color=transparent_white) for c in var_colors]

# do the image plot
p =figure(plot_width=900, plot_height = 900,
         tooltips = [
    ("x", "$x"),
    ("y", "$y"),
    ("bare","@BS"),
    ("dry", "NPV"),
    ("green","PV"),
    ("wet","@wet"),
    ("water","@water")], )

for i in range(5):
    p.image(image=image_list[i:i+1], x=fc_wofs_data.x.data.min(), y=fc_wofs_data.y.data.max(),
        dh=(fc_wofs_data.x.data.max() - fc_wofs_data.x.data.min()),
        dw=(fc_wofs_data.y.data.max() - fc_wofs_data.y.data.min()),
        color_mapper = color_map[i])
# to do
# legend, 
#title, 
#tooltip to show rignt value bla...
#arcgis outputs covered
show(p)

In [ ]:
# save spatial WIT as geotiff
# each variable will be output to individual COG
# file_name works as prefix, the final output file name will be "file_name_bandname", e.g. "test_BS.tif"
file_name = "test2"

save_geotiff(fc_wofs_perc, file_name)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** June 16 2020

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)

**Tags**: :index:`no_testing`,:index:`NCI compatible`,:index:`landsat 5`, :index:`landsat 7`,  :index:`landsat 8`, :index:`dea_plotting`, :index:`time series`